In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style

import torch 
from torch import nn

from torchsummary import summary

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [16]:
class block(nn.Module):
    def __init__(self, in_channels, num_conv, out_channels, max_pooling=True):
        super(block, self).__init__()
        all_convs = []
        for i in range(num_conv):
            if i==0: # just for changing numbe of channels
                all_convs.append(nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                              kernel_size=(3,3), padding=(1,1), stride=(1,1)))
            else: 
                all_convs.append(nn.Conv2d(in_channels=out_channels, out_channels=out_channels,  
                                              kernel_size=(3,3), padding=(1,1), stride=(1,1)))
            all_convs.append(nn.ReLU())
        all_convs.append(nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)))
        self.conv = nn.Sequential(*all_convs)

    def forward(self, x):
        return self.conv(x)

In [17]:
model = block(3, 1, 64, True).to(device)
summary(model, (3, 2, 2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 64, 2, 2]           1,792
              ReLU-2             [-1, 64, 2, 2]               0
         MaxPool2d-3             [-1, 64, 1, 1]               0
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [27]:
class VGG(nn.Module):
    def __init__(self, input_shape, layers, num_classes):
        # layers must be a list of tuples consisting of num of covolutions in one block and channels in 
        # that convolution
        # input_shape = (num_channels, height, width) 
        
        super(VGG, self).__init__()
        in_channels, height, width = input_shape
        all_convs = []
        for (num_conv, out_channels) in layers:
            #print(f"num_conv = {num_conv}, in_channels = {in_channels}, out_channels = {out_channels}")
            all_convs.append(block(in_channels=in_channels, num_conv=num_conv,
                                             out_channels=out_channels))
            in_channels = out_channels
        
        self.conv = nn.Sequential(*all_convs)
        height = height//(2**len(layers))
        width = width//(2**len(layers))
        self.fc1 = nn.Linear(in_features=layers[-1][1]*height*width, out_features=4096)    
        self.fc2 = nn.Linear(in_features = 4096, out_features=4096)
        self.fc3 = nn.Linear(in_features=4096, out_features=num_classes)
        self.dropout = nn.Dropout()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = nn.Flatten()(x)
        
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = self.fc3(x)
        return x

In [28]:
layers11 = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
input_shape = (3, 224, 224)
num_classes = 10
vgg11 = VGG(input_shape=input_shape, layers=layers11, num_classes=num_classes).to(device)
summary(vgg11, input_shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
             block-4         [-1, 64, 112, 112]               0
            Conv2d-5        [-1, 128, 112, 112]          73,856
              ReLU-6        [-1, 128, 112, 112]               0
         MaxPool2d-7          [-1, 128, 56, 56]               0
             block-8          [-1, 128, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         295,168
             ReLU-10          [-1, 256, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         590,080
             ReLU-12          [-1, 256, 56, 56]               0
        MaxPool2d-13          [-1, 256, 28, 28]               0
            block-14          [-1, 256,

In [29]:
layers19 = ((2, 64), (2, 128), (4, 256), (4, 512), (4, 512))
input_shape = (3, 224, 224)
num_classes = 10
vgg19 = VGG(input_shape=input_shape, layers=layers19, num_classes=num_classes).to(device)
summary(vgg19, input_shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
             block-6         [-1, 64, 112, 112]               0
            Conv2d-7        [-1, 128, 112, 112]          73,856
              ReLU-8        [-1, 128, 112, 112]               0
            Conv2d-9        [-1, 128, 112, 112]         147,584
             ReLU-10        [-1, 128, 112, 112]               0
        MaxPool2d-11          [-1, 128, 56, 56]               0
            block-12          [-1, 128, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         295,168
             ReLU-14          [-1, 256,